In [3]:
import numpy as np
from numba import njit

## nb_permutations

In [4]:
@njit(error_model="numpy")
def nb_permutations(pattern, length):
    # pattern :1d ndarray, likely np.arange(1,10,2)
    # length :int, length<=pattern.shape[0]
    loop = 1
    n_pattern = pattern.shape[0]
    for i in range(length):
        loop *= n_pattern - i
    return_arr = np.zeros((loop, length), dtype="int8")
    list_index_pattern = np.arange(n_pattern)
    for i in range(loop):
        mask = np.ones(n_pattern, dtype="bool")
        num = i
        for j in range(length):
            index_pattern = (list_index_pattern[mask])[num % (n_pattern - j)]
            return_arr[i, j] = pattern[index_pattern]
            num //= n_pattern - j
            mask[index_pattern] = False
    return return_arr

In [5]:
nb_permutations(np.arange(1,8,2), 2)

array([[1, 3],
       [3, 1],
       [5, 1],
       [7, 1],
       [1, 5],
       [3, 5],
       [5, 3],
       [7, 3],
       [1, 7],
       [3, 7],
       [5, 7],
       [7, 5]], dtype=int8)

## make_dict_change_x_pattern

In [6]:
def make_dict_change_x_pattern(max_ops):
    dict_change_x_pattern = dict()
    dict_change_x_pattern[0] = np.zeros((1, 1), dtype="int8")
    dict_max_loop = np.ones((max_ops + 1, max_ops + 1), dtype="int64")
    for i in range(1, max_ops + 1):
        change_x_pattern = nb_permutations(np.arange(1, max_ops + 2), i)  # not use 0,it is const(1)
        max_drop_num = np.zeros(change_x_pattern.shape[0], dtype="int8")
        for j in range(change_x_pattern.shape[0]):
            max_ = change_x_pattern[j, 0]
            save = max_ - 1
            for k in range(1, change_x_pattern.shape[1]):
                if max_ + 1 == change_x_pattern[j, k]:
                    max_ = change_x_pattern[j, k]
                elif max_ + 1 < change_x_pattern[j, k]:
                    max_ = change_x_pattern[j, k]
                    save = max_ - 1
            max_drop_num[j] = save
        dict_change_x_pattern[i] = change_x_pattern[np.argsort(max_drop_num)].astype("int8")
        dict_max_loop[i] = np.array([np.sum(max_drop_num <= j) for j in range(np.max(max_drop_num) + 1)])
    return dict_change_x_pattern, dict_max_loop

In [7]:
dict_change_x_pattern, dict_max_loop=make_dict_change_x_pattern(4)
dict_change_x_pattern

{0: array([[0]], dtype=int8),
 1: array([[1],
        [2],
        [3],
        [4],
        [5]], dtype=int8),
 2: array([[1, 2],
        [2, 1],
        [2, 3],
        [3, 1],
        [1, 3],
        [3, 2],
        [3, 4],
        [4, 1],
        [4, 2],
        [4, 5],
        [1, 4],
        [2, 4],
        [4, 3],
        [3, 5],
        [2, 5],
        [5, 2],
        [5, 3],
        [5, 1],
        [1, 5],
        [5, 4]], dtype=int8),
 3: array([[1, 2, 3],
        [2, 1, 3],
        [2, 3, 4],
        [2, 3, 1],
        [1, 3, 4],
        [3, 4, 5],
        [3, 2, 4],
        [3, 4, 1],
        [3, 4, 2],
        [3, 1, 4],
        [3, 2, 1],
        [1, 3, 2],
        [3, 1, 2],
        [4, 1, 3],
        [1, 4, 3],
        [4, 5, 3],
        [4, 2, 3],
        [4, 2, 5],
        [4, 5, 2],
        [1, 4, 5],
        [4, 3, 2],
        [2, 4, 3],
        [1, 2, 4],
        [4, 1, 5],
        [2, 4, 5],
        [4, 5, 1],
        [4, 3, 5],
        [4, 1, 2],
        [4, 3, 1

In [8]:
dict_max_loop

array([[  1,   1,   1,   1,   1],
       [  1,   2,   3,   4,   5],
       [  1,   3,   7,  13,  20],
       [  1,   4,  13,  33,  60],
       [  1,   5,  20,  60, 120]])

In [9]:
n=3
print(f"0 : {dict_change_x_pattern[n][:dict_max_loop[n,0]]}")
for i in range(1,dict_max_loop.shape[0]):
    print(f"{i} : {dict_change_x_pattern[n][dict_max_loop[n,i-1]]}~{dict_change_x_pattern[n][dict_max_loop[n,i]-1]}")

0 : [[1 2 3]]
1 : [2 1 3]~[2 3 1]
2 : [1 3 4]~[3 1 2]
3 : [4 1 3]~[4 2 1]
4 : [1 3 5]~[5 4 3]


## make_change_x_id

In [9]:
def make_change_x_id(mask, x_max):
    # mask -> [3,4,2] みたいな
    # x_max -> 全体としての最大値
    TF = np.ones(x_max+1, dtype="bool")
    return_num = 0
    for i in range(mask.shape[0]):
        return_num *= x_max - i
        return_num += np.sum(TF[: mask[i]])
        TF[mask[i]] = False
    return return_num

## make_dict_mask_x

In [10]:
def make_dict_mask_x(eq_x_max):
    dict_mask_x = dict()
    for i in range(eq_x_max + 1):
        __mask_x = nb_permutations(np.arange(1, i + 1), i)  # 第一要素が0にする。
        mask_x = np.zeros((__mask_x.shape[0], __mask_x.shape[1] + 1), dtype="int8")
        mask_x[:, 1:] = __mask_x  # 第一要素が0にする。
        change_x_num = np.array([make_change_x_id(mask_x[k, 1:], i) for k in range(mask_x.shape[0])])
        index = np.argsort(change_x_num)
        dict_mask_x[i] = mask_x[index].copy()
    return dict_mask_x
make_dict_mask_x(3)

{0: array([[0]], dtype=int8),
 1: array([[0, 1]], dtype=int8),
 2: array([[0, 1, 2],
        [0, 2, 1]], dtype=int8),
 3: array([[0, 1, 2, 3],
        [0, 1, 3, 2],
        [0, 2, 1, 3],
        [0, 2, 3, 1],
        [0, 3, 1, 2],
        [0, 3, 2, 1]], dtype=int8)}

In [11]:
eq_x_max=3
dict_mask_x=make_dict_mask_x(eq_x_max)
for i in range(eq_x_max+1):
    for j in range(dict_mask_x[i].shape[0]):
        print(f"{i}: {make_change_x_id(dict_mask_x[i][j,1:],i)}:{dict_mask_x[i][j]}")

0: 0:[0]
1: 1:[0 1]
2: 2:[0 1 2]
2: 3:[0 2 1]
3: 4:[0 1 2 3]
3: 5:[0 1 3 2]
3: 6:[0 2 1 3]
3: 7:[0 2 3 1]
3: 8:[0 3 1 2]
3: 9:[0 3 2 1]
